# Word2Vec을 활용한 모델 구현

In [0]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [0]:
DATA_IN_PATH = r'./data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH+TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiment = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

# word2vec 벡터화

In [0]:
## 학습시 필요한 하이퍼 파라미터
num_features = 300     # 각 단어에 대해 임베딩된 벡터의 차원
min_word_count = 40    # 단어에 대한 최소 빈도 수
num_workers = 4        # 학습을 위한 프로세스 개수
context = 10           # word2vec을 수행하기 위한 컨텍스트 윈도우 크기
downsampling = 1e-3    # 학습을 수행할 때 빠른 학습을 위한 정답 단어 라벨에 대한 다운샘플링 비율. 보통 0.001이 좋은 성능을 낸다.

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window = context,
                          sample = downsampling)

2019-07-31 12:39:06,995 : INFO : collecting all words and their counts
2019-07-31 12:39:07,004 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-07-31 12:39:07,400 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2019-07-31 12:39:07,727 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2019-07-31 12:39:07,896 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2019-07-31 12:39:07,897 : INFO : Loading a fresh vocabulary
2019-07-31 12:39:07,962 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2019-07-31 12:39:07,963 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2019-07-31 12:39:08,015 : INFO : deleting the raw counts dictionary of 74065 items
2019-07-31 12:39:08,018 : INFO : sample=0.001 downsamples 30 most-common words
2019-07-31 12:39:08,020 : INFO : downsampling leaves estimated 2494384 word corpus (94.9% of prior 2627273)
2019-07-31 12:39:08,059 : INFO : estimated required memory for 8160 words and 300 dimen

In [0]:
# 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담는다.
# 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다.
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-07-31 12:41:53,619 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-07-31 12:41:53,620 : INFO : not storing attribute vectors_norm
2019-07-31 12:41:53,622 : INFO : not storing attribute cum_table
2019-07-31 12:41:53,623 : WARNING : this function is deprecated, use smart_open.open instead
2019-07-31 12:41:53,856 : INFO : saved 300features_40minwords_10context


In [0]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [0]:
def get_dataset(reviews, model, num_features):
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
    
    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [0]:
train_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


# 학습과 검증  데이터셋 분리

In [0]:
from sklearn.model_selection import train_test_split

X = train_data_vecs
y = np.array(sentiment)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

# 모델 선언 및 학습

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

# 검증 데이터로 성능 평가

In [0]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.863000


# 캐글에 데이터 제출

In [0]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH+TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [0]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [0]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [0]:
DATA_OUT_PATH = r'./data_out/'

test_predicted = lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH+'lgs_w2v_answer.csv', index=False, quoting=3)